In [1]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from multiprocess import Pool

from tls_paper_funcs import cert_size_inference_simple_diagram
from tls_paper_funcs import tls_12_appdata_filtering, remove_empty

from tls_paper_funcs import handshake_removal_only
tls_dir = ['tls_dir_'+str(x) for x in range(20)]
tls_bs = ['tls_b_'+str(x) for x in range(20)]

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
#df = pd.read_csv('data/tranco/dataset.csv', engine='python')
df = pd.read_csv('processed-datasets/tranco-dataset.csv', engine='python')
df.describe()

,c_port,s_port,ip_proto,start_time,c_tls_version,s_tls_version,c_supported_versions,s_supported_versions,c_psk_present,c_psk_kem,s_psk,hello_retry_req,tls_b_0,tls_b_1,tls_b_2,tls_b_3,tls_b_4,tls_b_5,tls_b_6,tls_b_7,tls_b_8,tls_b_9,tls_b_10,tls_b_11,tls_b_12,tls_b_13,tls_b_14,tls_b_15,tls_b_16,tls_b_17,tls_b_18,tls_b_19,tls_ipt_0,tls_ipt_1,tls_ipt_2,tls_ipt_3,tls_ipt_4,tls_ipt_5,tls_ipt_6,tls_ipt_7,tls_ipt_8,tls_ipt_9,tls_ipt_10,tls_ipt_11,tls_ipt_12,tls_ipt_13,tls_ipt_14,tls_ipt_15,tls_ipt_16,tls_ipt_17,tls_ipt_18,tls_ipt_19,tls_dir_0,tls_dir_1,tls_dir_2,tls_dir_3,tls_dir_4,tls_dir_5,tls_dir_6,tls_dir_7,tls_dir_8,tls_dir_9,tls_dir_10,tls_dir_11,tls_dir_12,tls_dir_13,tls_dir_14,tls_dir_15,tls_dir_16,tls_dir_17,tls_dir_18,tls_dir_19,tls_tp_0,tls_tp_1,tls_tp_2,tls_tp_3,tls_tp_4,tls_tp_5,tls_tp_6,tls_tp_7,tls_tp_8,tls_tp_9,tls_tp_10,tls_tp_11,tls_tp_12,tls_tp_13,tls_tp_14,tls_tp_15,tls_tp_16,tls_tp_17,tls_tp_18,tls_tp_19,cert_size_guess_simple,total_s_cert
count,505938.000000,505938.000000,505938.0,5.059380e+05,505938.000000,505938.000000,505938.00000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.00000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,505938.000000,219584.000000,505938.000000
mean,46840.186041,480.451034,6.0,1.706651e+09,4.997474,4.349039,10956.41499,133.866911,0.009305,54.043405,-0.991841,-0.106954,415.603317,142.600556,1694.680232,1255.527407,876.730040,139.088499,104.406465,305.021930,679.028278,579.554283,327.301553,387.389283,503.856419,484.663637,474.728154,504.562818,568.707268,625.764855,618.934879,560.802994,-0.000004,267.388860,2.774271,6.476983,11.478484,12.519959,17.214748,57.002498,77.598423,75.602129,69.909477,104.258289,66.189780,64.188683,57.615605,65.665058,49.088754,38.806482,30.659662,23.861013,0.109009,0.791548,0.786934,0.783910,0.518907,0.287183,0.058549,0.020520,0.242127,0.342961,0.347464,0.113763,0.009804,-0.11973,-0.197145,-0.298900,-0.437706,-0.503435,-0.546990,-0.585605,22.005354,19.706343,18.798758,20.137191,19.227030,19.640845,18.810289,18.658395,20.322308,19.600141,17.856067,16.037588,15.239438,13.554445,12.167689,9.719857,7.346345,6.385008,5.688897,5.060998,3996.849966,1700.834241
std,8251.407080,1347.968123,0.0,2.946725e+04,0.123084,1.866018,10120.55189,151.477290,0.096015,50.839512,0.089959,0.313997,209.521865,545.429155,2083.477880,1994.779616,2052.861061,1544.618004,1254.810308,2808.309832,4756.971130,4309.460991,3083.770727,3267.197221,3386.566060,3130.865715,3081.082118,3108.849511,3229.331249,3288.907039,3239.076964,3036.550783,0.002812,297.834237,75.364925,115.217129,155.889290,135.773884,236.249932,249.360035,266.036680,311.812295,484.941264,651.830009,511.490512,658.715712,490.349518,469.828286,443.254025,388.007168,369.267554,337.638042,0.311670,0.603414,0.608051,0.610784,0.688220,0.677415,0.584353,0.575308,0.712830,0.752368,0.812178,0.855816,0.846526,0.83106,0.839301,0.877154,0.849495,0.818925,0.796871,0.773760,0.080695,

In [3]:
df['tls_dir_19'].value_counts()

tls_dir_19
-1    386342
 1     90064
 0     29531
-2         1
Name: count, dtype: int64

In [4]:
386342 / 505938

0.7636153046420707

In [5]:
df['total_s_cert'] = df['s_cert_lengths'].map(lambda x: sum(map(int,str(x).split(':'))))

## Certificate Size Inference
This was already performed for the `dataset.csv` file, which was then overwritten.  
This can be done again, but it takes some time.

In [6]:
tls13samples = list(df[(df['tls_version_guess'] == 'TLS 1.3') & (df['s_psk'] == -1)].index)

In [8]:
_ = """
def cert_size_guess_parallel(row_id):
    return cert_size_inference_simple_diagram(df.loc[row_id], row_id), row_id

with Pool() as p:
    pool_outputs = list(
        tqdm(
            p.imap(cert_size_guess_parallel, tls13samples),
            total=len(tls13samples)
        )
    )

for cert_guess, row_id in pool_outputs:
    df.loc[row_id, 'cert_size_guess_simple'] = cert_guess
"""

In [9]:
df.loc[tls13samples]['cert_size_guess_simple'].describe()

count    219583.000000
mean       3996.868173
std        1372.872682
min          -2.000000
25%        3226.000000
50%        4247.000000
75%        4677.500000
max       47702.000000
Name: cert_size_guess_simple, dtype: float64

In [10]:
df[df['tls_version_guess']=='TLS 1.2']['total_s_cert'].describe()

count    282227.000000
mean       3049.815875
std        1826.488718
min          -1.000000
25%        2304.000000
50%        3855.000000
75%        4272.500000
max       10792.000000
Name: total_s_cert, dtype: float64

Do we always get the same certificate size for the same SNIs?

In [11]:
def sni_check_parallel(sni):
    cert_size = df[(df['sni'] == sni) & (df['total_s_cert'] > -1)]['total_s_cert'].unique()
    if len(cert_size) > 1:
        return (sni, cert_size)
    return ()

In [12]:
# Check SNIs that correspond to multiple different certificate sizes **in TLS 1.2**.
# This should be uncommon, but it might happen. Since it is TLS 1.2 (cleartext certificates),
# these flows can be manually checked to better understant what's happening.
with Pool() as p:
    sni_outputs = list(
        tqdm(
            p.imap(sni_check_parallel, list(df['sni'].unique())),
            total=len(list(df['sni'].unique()))
        )
    )

  0%|          | 0/77038 [00:00<?, ?it/s]

In [13]:
[k for k in sni_outputs if len(k)>0]

[('news.google.com', array([4003, 2621])),
 ('k.sinaimg.cn', array([4988, 3678])),
 ('www.googletagmanager.com', array([4460, 3078, 1644])),
 ('stats.g.doubleclick.net', array([4226, 2844, 1410])),
 ('static.facebook.com', array([2896, 2897])),
 ('connect.facebook.net', array([2896, 2897])),
 ('munchkin.marketo.net', array([2943, 2675])),
 ('fonts.gstatic.com', array([4012, 2630])),
 ('browser.events.data.microsoft.com', array([3977, 3979, 3976])),
 ('accounts.google.com', array([4010, 4013])),
 ('api.ipify.org', array([5633, 4555])),
 ('web.facebook.com', array([2896, 2897])),
 ('static.xx.fbcdn.net', array([2896, 2897])),
 ('facebook.com', array([2896, 2897])),
 ('www.facebook.com', array([2896, 2897])),
 ('acdn.adnxs.com', array([3102, 3178])),
 ('cdn.jsdelivr.net', array([2819, 1648])),
 ('res.cloudinary.com', array([4136, 4137])),
 ('www.youtube.com', array([6518, 3702])),
 ('service.force.com', array([3475, 3476, 3477])),
 ('fastlane.rubiconproject.com', array([2967, 4716])),
 ('

In [14]:
# What do Tranco's small certificates look like?
df[(df['total_s_cert'] > -1)].sort_values('total_s_cert')[:100]

,c_ip,c_port,s_ip,s_port,ip_proto,start_time,c_tls_version,s_tls_version,c_supported_versions,s_supported_versions,c_psk_present,c_psk_kem,s_psk,hello_retry_req,tls_b_0,tls_b_1,tls_b_2,tls_b_3,tls_b_4,tls_b_5,tls_b_6,tls_b_7,tls_b_8,tls_b_9,tls_b_10,tls_b_11,tls_b_12,tls_b_13,tls_b_14,tls_b_15,tls_b_16,tls_b_17,tls_b_18,tls_b_19,tls_ipt_0,tls_ipt_1,tls_ipt_2,tls_ipt_3,tls_ipt_4,tls_ipt_5,tls_ipt_6,tls_ipt_7,tls_ipt_8,tls_ipt_9,tls_ipt_10,tls_ipt_11,tls_ipt_12,tls_ipt_13,tls_ipt_14,tls_ipt_15,tls_ipt_16,tls_ipt_17,tls_ipt_18,tls_ipt_19,tls_dir_0,tls_dir_1,tls_dir_2,tls_dir_3,tls_dir_4,tls_dir_5,tls_dir_6,tls_dir_7,tls_dir_8,tls_dir_9,tls_dir_10,tls_dir_11,tls_dir_12,tls_dir_13,tls_dir_14,tls_dir_15,tls_dir_16,tls_dir_17,tls_dir_18,tls_dir_19,tls_tp_0,tls_tp_1,tls_tp_2,tls_tp_3,tls_tp_4,tls_tp_5,tls_tp_6,tls_tp_7,tls_tp_8,tls_tp_9,tls_tp_10,tls_tp_11,tls_tp_12,tls_tp_13,tls_tp_14,tls_tp_15,tls_tp_16,tls_tp_17,tls_tp_18,tls_tp_19,sni,s_cert_lengths,source,file,tls_version_guess,cert_size_guess_simple,total_s_cert
279643,172.19.22.7,39386,107.178.223.183,443,6,1.706654e+09,5,5,-1,-1,0,-1,-1,0,184,59,329,114,4,37,1,40,26,139,1,40,26,-1,-1,-1,-1,-1,-1,-1,0,301,0,0,0,0,0,0,1,218,0,0,120,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,1,0,0,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,22,22,22,22,22,22,20,22,21,22,20,22,21,-1,-1,-1,-1,-1,-1,-1,ne16.biz,319,tranco,shuf_tranco207_tls12_2024_01_30_22h36,TLS 1.2,NaN,319
375815,172.19.22.2,41312,107.178.223.183,443,6,1.706676e+09,5,5,-1,-1,0,-1,-1,0,185,59,330,115,4,37,1,40,26,139,1,40,26,-1,-1,-1,-1,-1,-1,-1,0,934,0,0,0,1,0,0,1,939,0,0,0,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,1,0,0,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,22,22,22,22,22,22,20,22,21,22,20,22,21,-1,-1,-1,-1,-1,-1,-1,procrd.vc,320,tranco,shuf_tranco406_tls12_2024_01_31_04h45,TLS 1.2,NaN,320
235203,172.19.22.2,60866,107.178.223.183,443,6,1.706669e+09,5,5,-1,-1,0,-1,-1,0,184,59,330,115,4,37,1,40,26,139,1,40,26,-1,-1,-1,-1,-1,-1,-1,0,179,0,0,0,1,0,0,1,161,0,0,0,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,1,0,0,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,22,22,22,22,22,22,20,22,21,22,20,22,21,-1,-1,-1,-1,-1,-1,-1,e-nt.net,320,tranco,shuf_tranco151_tls12_2024_01_31_02h44,TLS 1.2,NaN,320
419400,172.19.22.2,56550,107.178.223.183,443,6,1.706703e+09,5,5,-1,-1,0,-1,-1,0,184,59,330,115,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,169,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,22,22,22,22,22,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,xmr5b.ru,320,tranco,shuf_tranco496_tls12_2024_01_31_12h02,TLS 1.2,NaN,320
473952,172.19.22.3,60760,104.155.138.21,443,6,1.706683e+09,5,5,-1,-1,0,-1,-1,0,185,59,330,115,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,8414,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,22,22,22,22,22,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,4bxct.com,320,tranco,shuf_tranco920_tls12_2024_01_31_06h42,TLS 1.2,NaN,320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246510,172.19.22.4,57048,80.93.53.97,443,6,1.706677e+09,5,5,-1,-1,0,-1,-1,0,188,80,689,205,4,70,1,40,26,202,1,40,-1,-1,-1,-1,-1,-1,-1,-1,0,97,0,0,0,0,0,0,0,160,0,0,-1,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,1,0,0,0,0,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,22,22,22,22,22,22,20,22,21,22,20,22,-1,-1,-1,-1,-1,-1,-1,-1,gigapeta.com,679,tranco,shuf_tranco165_tls12_2024_01_31_04h48,TLS 1.2,NaN,679
481723,172.19.22.2,47610,51.38.84.174,443,6,1.706689e+09,5,5,-1,-1,0,-1,-1,0,189,106,692,300,4,37,1,40,26,1,40,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,64,0,0,0,0,0,0,0,50,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,1,0,0,0,0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,22,22,22,22,22,22,20,22,21,20,22,-1,-1,-1,-1,-1,-1,-1,-

In [15]:
# Here, the files get overwritten:
#df.to_csv('data/tranco/dataset.csv', index=False)
#df.to_csv('processed-datasets/tranco-dataset.csv', index=False)

# BEHAV

In [16]:
#print(list(df.columns))
id_cols = ['c_ip', 'c_port', 's_ip', 's_port', 'ip_proto', 'start_time', 'c_tls_version', 's_tls_version', 'c_supported_versions', 's_supported_versions', 'c_psk_present', 'c_psk_kem', 's_psk', 'hello_retry_req', 'sni', 's_cert_lengths', 'source', 'file', 'tls_version_guess', 'total_s_cert']

In [17]:
tlsdata12 = tls_12_appdata_filtering(df[df['tls_version_guess'] == 'TLS 1.2'])
remove_empty(tlsdata12, True)
tlsdata12[id_cols] = df[id_cols]
tlsdata12

81549


,tls_b_0,tls_b_1,tls_b_2,tls_b_3,tls_b_4,tls_b_5,tls_b_6,tls_b_7,tls_b_8,tls_b_9,tls_b_10,tls_b_11,tls_b_12,tls_b_13,tls_b_14,tls_b_15,tls_b_16,tls_b_17,tls_b_18,tls_b_19,tls_dir_0,tls_dir_1,tls_dir_2,tls_dir_3,tls_dir_4,tls_dir_5,tls_dir_6,tls_dir_7,tls_dir_8,tls_dir_9,tls_dir_10,tls_dir_11,tls_dir_12,tls_dir_13,tls_dir_14,tls_dir_15,tls_dir_16,tls_dir_17,tls_dir_18,tls_dir_19,c_ip,c_port,s_ip,s_port,ip_proto,start_time,c_tls_version,s_tls_version,c_supported_versions,s_supported_versions,c_psk_present,c_psk_kem,s_psk,hello_retry_req,sni,s_cert_lengths,source,file,tls_version_guess,total_s_cert
0,64.0,189.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,172.19.22.2,57044,1.0.0.3,443,6,1.706618e+09,5,5,-1,-1,0,-1,-1,0,family.cloudflare-dns.com,1654:1272,tranco,shuf_tranco000_tls12_2024_01_30_12h40,TLS 1.2,2926
1,64.0,189.0,33.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,172.19.22.2,57072,1.0.0.3,443,6,1.706618e+09,5,5,-1,-1,0,-1,-1,0,family.cloudflare-dns.com,1654:1272,tranco,shuf_tranco000_tls12_2024_01_30_12h40,TLS 1.2,2926
3,172.0,63.0,33.0,33.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,172.19.22.2,58110,23.47.188.208,443,6,1.706618e+09,5,5,-1,-1,0,-1,-1,0,www.t-mobile.com,1730:1298:1090,tranco,shuf_tranco000_tls12_2024_01_30_12h40,TLS 1.2,4118
4,438.0,361.0,250.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,172.19.22.2,50672,15.72.80.135,443,6,1.706618e+09,5,5,-1,-1,0,-1,-1,0,hp.com,2264:1228:914,tranco,shuf_tranco000_tls12_2024_01_30_12h40,TLS 1.2,4406
5,64.0,172.0,306.0,33.0,33.0,270.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,0.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,172.19.22.2,38852,108.157.109.9,443,6,1.706618e+09,5,5,-1,-1,0,-1,-1,0,huffpost.com,1745:1122:1174:1145,tranco,shuf_tranco000_tls12_2024_01_30_12h40,TLS 1.2,5186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505209,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,172.19.22.3,41696,1.1.1.3,443,6,1.706543e+09,5,-1,-1,-1,0,-1,-1,-1,NaN,-1,tranco,tranco001_tls13_2024_01_29_14h59,TLS 1.2,-1
505321,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,172.19.22.3,41696,1.1.1.3,443,6,1.706544e+09,5,-1,-1,-1,0,-1,-1,-1,NaN,-1,tranco,tranco001_tls13_2024_01_29_14h59,TLS 1.2,-1
505413,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,51.0,154.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,172.19.22.3,41696,1.1.1.3,443,6,1.706544e+09,5,-1,-1,-1,0,-1,-1,-1,NaN,-1,tranco,tranco001_tls13_2024_01_29_14h59,TLS 1.2,-1
505535,51.0,154.0,51.0,154.0,376.0,494.0,26.0,274.0,494.0,26.0,51.0,154.0,51.0,154.0,51.0,154.0,295.0,494.0,26.0,277.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,172.19.22.3,41696,1.1.1.3,443,6,1.706544e+09,5,5,-1,-1,0,-1,-1,0,NaN,-1,tranco,tranco001_tls13_2024_01_29_14h59,TLS 1.2,-1


In [18]:
#tlsdata12.to_csv('processed-datasets/tranco-tls12-data.csv')

In [19]:
## tls1.3:
tranco13_filtered = tls_12_appdata_filtering(df[df['tls_version_guess'] == 'TLS 1.3']).astype(int)
remove_empty(tranco13_filtered, True)
tranco13_filtered

27


,tls_b_0,tls_b_1,tls_b_2,tls_b_3,tls_b_4,tls_b_5,tls_b_6,tls_b_7,tls_b_8,tls_b_9,tls_b_10,tls_b_11,tls_b_12,tls_b_13,tls_b_14,tls_b_15,tls_b_16,tls_b_17,tls_b_18,tls_b_19,tls_dir_0,tls_dir_1,tls_dir_2,tls_dir_3,tls_dir_4,tls_dir_5,tls_dir_6,tls_dir_7,tls_dir_8,tls_dir_9,tls_dir_10,tls_dir_11,tls_dir_12,tls_dir_13,tls_dir_14,tls_dir_15,tls_dir_16,tls_dir_17,tls_dir_18,tls_dir_19
19848,3170,53,182,461,66,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
19849,3170,53,182,461,66,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
19850,5124,53,165,445,66,26,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
19851,5124,53,182,445,66,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
19853,3171,53,461,57,165,145,154,26,51,154,51,154,26,392,494,26,-1,-1,-1,-1,1,0,1,1,0,0,0,0,0,0,0,0,1,1,1,1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505933,3949,53,182,182,53,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
505934,3949,53,182,182,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
505935,3949,53,182,445,57,26,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,1,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
505936,3949,53,182,182,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [20]:
tranco13_filtered[id_cols] = df[id_cols]
tranco13_filtered

,tls_b_0,tls_b_1,tls_b_2,tls_b_3,tls_b_4,tls_b_5,tls_b_6,tls_b_7,tls_b_8,tls_b_9,tls_b_10,tls_b_11,tls_b_12,tls_b_13,tls_b_14,tls_b_15,tls_b_16,tls_b_17,tls_b_18,tls_b_19,tls_dir_0,tls_dir_1,tls_dir_2,tls_dir_3,tls_dir_4,tls_dir_5,tls_dir_6,tls_dir_7,tls_dir_8,tls_dir_9,tls_dir_10,tls_dir_11,tls_dir_12,tls_dir_13,tls_dir_14,tls_dir_15,tls_dir_16,tls_dir_17,tls_dir_18,tls_dir_19,c_ip,c_port,s_ip,s_port,ip_proto,start_time,c_tls_version,s_tls_version,c_supported_versions,s_supported_versions,c_psk_present,c_psk_kem,s_psk,hello_retry_req,sni,s_cert_lengths,source,file,tls_version_guess,total_s_cert
19848,3170,53,182,461,66,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.4,55296,1.1.1.3,443,6,1.706620e+09,5,5,20304,304,0,101,-1,0,family.cloudflare-dns.com,-1,tranco,shuf_tranco002_tls13_2024_01_30_13h04,TLS 1.3,-1
19849,3170,53,182,461,66,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.4,55306,1.1.1.3,443,6,1.706620e+09,5,5,20304,304,0,101,-1,0,family.cloudflare-dns.com,-1,tranco,shuf_tranco002_tls13_2024_01_30_13h04,TLS 1.3,-1
19850,5124,53,165,445,66,26,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.4,47308,104.21.2.190,443,6,1.706620e+09,5,5,20304,304,0,101,-1,0,wl5.peachavocado.com,-1,tranco,shuf_tranco002_tls13_2024_01_30_13h04,TLS 1.3,-1
19851,5124,53,182,445,66,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.4,47320,104.21.2.190,443,6,1.706620e+09,5,5,20304,304,0,101,-1,0,wl5.peachavocado.com,-1,tranco,shuf_tranco002_tls13_2024_01_30_13h04,TLS 1.3,-1
19853,3171,53,461,57,165,145,154,26,51,154,51,154,26,392,494,26,-1,-1,-1,-1,1,0,1,1,0,0,0,0,0,0,0,0,1,1,1,1,-1,-1,-1,-1,172.19.22.4,55302,1.1.1.3,443,6,1.706620e+09,5,5,20304,304,0,101,-1,0,family.cloudflare-dns.com,-1,tranco,shuf_tranco002_tls13_2024_01_30_13h04,TLS 1.3,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505933,3949,53,182,182,53,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.3,32934,104.22.56.216,443,6,1.706544e+09,5,5,20304,304,0,101,-1,0,static.cardmarket.com,-1,tranco,tranco001_tls13_2024_01_29_14h59,TLS 1.3,-1
505934,3949,53,182,182,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.3,32942,104.22.56.216,443,6,1.706544e+09,5,5,20304,304,0,101,-1,0,static.cardmarket.com,-1,tranco,tranco001_tls13_2024_01_29_14h59,TLS 1.3,-1
505935,3949,53,182,445,57,26,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,1,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.3,32958,104.22.56.216,443,6,1.706544e+09,5,5,20304,304,0,101,-1,0,static.cardmarket.com,-1,tranco,tranco001_tls13_2024_01_29_14h59,TLS 1.3,-1
505936,3949,53,182,182,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.3,32968,104.22.56.216,443,6,1.706544e+09,5,5,20304,304,0,101,-1,0,static.cardmarket.com,-1,tranco,tranco001_tls13_2024_01_29_14h59,TLS 1.3,-1


In [21]:
#tranco13_filtered.to_csv('processed-datasets/tranco-tls13-data.csv')

In [22]:
def tls_hs_remove_parallel(i):
    nst_discard_b, nst_discard_dir = handshake_removal_only(tranco13_filtered.loc[i])
    line = np.hstack( [nst_discard_b,   (20 - len(nst_discard_b))*[-1],
                       nst_discard_dir, (20 - len(nst_discard_dir))*[-1]] )
    return (i,line)

In [23]:
tls13samples = list(tranco13_filtered.index)
with Pool() as p:
    tls13_appdata_no_hs = list(
        tqdm(
            p.imap(tls_hs_remove_parallel, tls13samples),
            total=len(tls13samples)
        )
    )

  0%|          | 0/223684 [00:00<?, ?it/s]

In [24]:
tranco13_no_hs_new = pd.DataFrame(np.vstack([b for a,b in tls13_appdata_no_hs]),
                            columns= tls_bs+tls_dir,
                            index=[a for a,b in tls13_appdata_no_hs])
to_remove13 = tranco13_no_hs_new[ (tranco13_no_hs_new[tls_bs] == -1).all(axis=1) ].index
print(len(to_remove13))
tranco13_no_hs_new.drop(to_remove13, inplace=True)
tranco13_no_hs_new[id_cols + ['cert_size_guess_simple']] = df[id_cols + ['cert_size_guess_simple']]
tranco13_no_hs_new

13823


,tls_b_0,tls_b_1,tls_b_2,tls_b_3,tls_b_4,tls_b_5,tls_b_6,tls_b_7,tls_b_8,tls_b_9,tls_b_10,tls_b_11,tls_b_12,tls_b_13,tls_b_14,tls_b_15,tls_b_16,tls_b_17,tls_b_18,tls_b_19,tls_dir_0,tls_dir_1,tls_dir_2,tls_dir_3,tls_dir_4,tls_dir_5,tls_dir_6,tls_dir_7,tls_dir_8,tls_dir_9,tls_dir_10,tls_dir_11,tls_dir_12,tls_dir_13,tls_dir_14,tls_dir_15,tls_dir_16,tls_dir_17,tls_dir_18,tls_dir_19,c_ip,c_port,s_ip,s_port,ip_proto,start_time,c_tls_version,s_tls_version,c_supported_versions,s_supported_versions,c_psk_present,c_psk_kem,s_psk,hello_retry_req,sni,s_cert_lengths,source,file,tls_version_guess,total_s_cert,cert_size_guess_simple
19848,182,66,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.4,55296,1.1.1.3,443,6,1.706620e+09,5,5,20304,304,0,101,-1,0,family.cloudflare-dns.com,-1,tranco,shuf_tranco002_tls13_2024_01_30_13h04,TLS 1.3,-1,3021.0
19849,182,66,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.4,55306,1.1.1.3,443,6,1.706620e+09,5,5,20304,304,0,101,-1,0,family.cloudflare-dns.com,-1,tranco,shuf_tranco002_tls13_2024_01_30_13h04,TLS 1.3,-1,3021.0
19850,165,66,26,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.4,47308,104.21.2.190,443,6,1.706620e+09,5,5,20304,304,0,101,-1,0,wl5.peachavocado.com,-1,tranco,shuf_tranco002_tls13_2024_01_30_13h04,TLS 1.3,-1,4975.0
19851,182,66,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.4,47320,104.21.2.190,443,6,1.706620e+09,5,5,20304,304,0,101,-1,0,wl5.peachavocado.com,-1,tranco,shuf_tranco002_tls13_2024_01_30_13h04,TLS 1.3,-1,4975.0
19853,57,165,145,154,26,51,154,51,154,26,392,494,26,-1,-1,-1,-1,-1,-1,-1,1,0,0,0,0,0,0,0,0,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,172.19.22.4,55302,1.1.1.3,443,6,1.706620e+09,5,5,20304,304,0,101,-1,0,family.cloudflare-dns.com,-1,tranco,shuf_tranco002_tls13_2024_01_30_13h04,TLS 1.3,-1,3022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505933,182,182,53,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.3,32934,104.22.56.216,443,6,1.706544e+09,5,5,20304,304,0,101,-1,0,static.cardmarket.com,-1,tranco,tranco001_tls13_2024_01_29_14h59,TLS 1.3,-1,3800.0
505934,182,182,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.3,32942,104.22.56.216,443,6,1.706544e+09,5,5,20304,304,0,101,-1,0,static.cardmarket.com,-1,tranco,tranco001_tls13_2024_01_29_14h59,TLS 1.3,-1,3800.0
505935,182,57,26,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.3,32958,104.22.56.216,443,6,1.706544e+09,5,5,20304,304,0,101,-1,0,static.cardmarket.com,-1,tranco,tranco001_tls13_2024_01_29_14h59,TLS 1.3,-1,3800.0
505936,182,182,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,172.19.22.3,32968,104.22.56.216,443,6,1.706544e+09,5,5,20304,304,0,101,-1,0,static.cardmarket.com,-1,tranco,tranco001_tls13_2024_01_29_14h59,TLS 1.3,-1,3800.0


In [25]:
tranco13_no_hs_new['tls_dir_0'].value_counts()

tls_dir_0
0    197320
1     12541
Name: count, dtype: int64

In [26]:
tranco13_no_hs_new[tls_bs].astype(float).describe()

,tls_b_0,tls_b_1,tls_b_2,tls_b_3,tls_b_4,tls_b_5,tls_b_6,tls_b_7,tls_b_8,tls_b_9,tls_b_10,tls_b_11,tls_b_12,tls_b_13,tls_b_14,tls_b_15,tls_b_16,tls_b_17,tls_b_18,tls_b_19
count,209861.000000,209861.000000,209861.000000,209861.000000,209861.000000,209861.000000,209861.000000,209861.000000,209861.000000,209861.000000,209861.000000,209861.000000,209861.000000,209861.000000,209861.000000,209861.000000,209861.000000,209861.0,209861.0,209861.0
mean,196.812538,352.827438,269.814167,263.941504,340.089002,555.828172,650.960793,608.981249,575.686211,366.158424,218.967402,158.672469,137.093957,3.933141,-0.323371,-0.994477,-0.995078,-1.0,-1.0,-1.0
std,269.682125,1468.943536,1634.108370,1680.391884,1929.229573,2393.055000,2527.674262,2404.293563,2326.306706,1706.406209,1064.567725,694.872615,567.795440,189.630078,64.669541,0.770738,0.717177,0.0,0.0,0.0
min,19.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0
25%,165.000000,57.000000,26.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0
50%,182.000000,165.000000,26.000000,26.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0
75%,182.000000,284.000000,57.000000,26.000000,26.000000,57.000000,93.000000,34.000000,34.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0
max,57379.000000,64895.000000,65106.000000,62921.000000,64901.000000,60561.000000,32197.000000,54913.000000,39899.000000,16401.000000,54796.000000,33538.000000,16401.000000,16401.000000,11397.000000,133.000000,121.000000,-1.0,-1.0,-1.0


In [27]:
#tranco13_no_hs_new.to_csv('processed-datasets/tranco-tls13-behav-new.csv')